In [1]:
import numpy as np
from eitprocessing.binreader import Sequence

In [2]:
file_path = '' #input path to test file 
FRAMERATE = 50 # Standard framerate
VENDOR = 'Timpel' # Vendor of EIT device used for data acquisition 

In [3]:
ignore_nan_columns = True
ignore_nan_rows = True

In [4]:
measurement = Sequence.from_path(path=file_path, vendor=VENDOR, framerate=FRAMERATE)

/Users/juliettefrancovich/Documents/KT/TM3/eitprocessing/eitprocessing/binreader/sequence.py:515: UserWarning: The number of frames requested (None) is largerthan the available number (180288) of frames after the first frame selected (0).
180288 frames have been loaded.
  warnings.warn(


In [5]:
short_section = measurement[80140:81600]

In [6]:
bc = short_section.framesets['raw'].deepcopy()
bc.name = 'baseline_corrected'
bc.description = 'raw data with a baseline correction for this section'
bc.pixel_values = bc.pixel_values - bc.pixel_baseline
bc.params.update({'pre-processing': 'pixel baseline correction'})
short_section.framesets['bc'] = bc

/Users/juliettefrancovich/Documents/KT/TM3/eitprocessing/eitprocessing/binreader/frameset.py:82: RuntimeWarning: All-NaN slice encountered
  return np.nanmin(self.pixel_values, axis=0)


In [7]:
pixel_values = short_section.framesets['bc'].pixel_values

In [8]:
matrix = pixel_values[300, :, :]

In [49]:
horizontal = True

In [50]:
n_groups = 4

In [51]:
## Test geometrical split

axis = 0 if horizontal else 1

# create a vector that is nan if the entire column/row is nan, 1 otherwise
vector_is_nan = np.all(np.isnan(matrix), axis=axis)
vector = np.ones(vector_is_nan.shape)

if (horizontal and ignore_nan_columns) or (
    not horizontal and ignore_nan_rows
):
    vector[vector_is_nan] = np.nan

    # remove non-numeric (nan) elements at vector ends
    # nan elements between numeric elements are kept
    numeric_element_indices = np.argwhere(~np.isnan(vector))
    first_num_element = numeric_element_indices.min()
    last_num_element = numeric_element_indices.max()
else:
    first_num_element = 0
    last_num_element = len(vector) - 1

n_elements = last_num_element - first_num_element + 1

group_size = n_elements / n_groups

if group_size < 1:
    if horizontal:
        print(f"The number of horizontal regions ({n_groups}) is larger than the number of available columns ({n_elements}).")
    else:
        print(f"The number of vertical regions ({n_groups}) is larger than the number of available columns ({n_elements}).")

# find the right boundaries (upper values) of each group
right_boundaries = (np.arange(n_groups) + 1) * group_size
right_boundaries = right_boundaries[:, np.newaxis]  # converts to row vector

# each row in the base represents one group
base = np.tile(np.arange(n_elements), (n_groups, 1))

# if the element number is higher than the split, it does not belong in this group
element_contribution_to_group = right_boundaries - base
element_contribution_to_group[element_contribution_to_group < 0] = 0

# if the element to the right is a full group size, this element is ruled out
rule_out = element_contribution_to_group[:, 1:] >= group_size
element_contribution_to_group[:, :-1][rule_out] = 0

# elements have a maximum value of 1
element_contribution_to_group = np.fmin(element_contribution_to_group, 1)

# if this element is already represented in the previous group (row), subtract that
element_contribution_to_group[1:] -= element_contribution_to_group[:-1]
element_contribution_to_group[element_contribution_to_group < 0] = 0

# element_contribution_to_group only represents non-nan elements
# insert into final including non-nan elements
final = np.full((n_groups, len(vector)), np.nan)
final[
    :, first_num_element : last_num_element + 1
] = element_contribution_to_group

# convert to list of vectors
final = [final[n, :] for n in range(final.shape[0])]


In [52]:
## Test physiological split

axis = 0 if horizontal else 1

# create a vector that is nan if the entire column/row is nan, 1 otherwise
vector_is_nan = np.all(np.isnan(matrix), axis=axis)
vector = np.ones(vector_is_nan.shape)

if (horizontal and ignore_nan_columns) or (
    not horizontal and ignore_nan_rows
):
    vector[vector_is_nan] = np.nan

    # remove non-numeric (nan) elements at vector ends
    # nan elements between numeric elements are kept
    numeric_element_indices = np.argwhere(~np.isnan(vector))
    first_num_element = numeric_element_indices.min()
    last_num_element = numeric_element_indices.max()
else:
    first_num_element = 0
    last_num_element = len(vector) - 1

n_elements = last_num_element - first_num_element + 1

group_size = n_elements / n_groups

if group_size < 1:
    if horizontal:
        print(f"The number of horizontal regions ({n_groups}) is larger than the number of available columns ({n_elements}).")
    else:
        print(f"The number of vertical regions ({n_groups}) is larger than the number of available columns ({n_elements}).")

sum_along_axis = np.nansum(matrix, axis=axis)
relative_sum_along_axis = sum_along_axis / np.nansum(matrix)
relative_cumsum_along_axis = np.cumsum(relative_sum_along_axis)

lower_bounds = np.arange(n_groups) / n_groups
upper_bounds = (np.arange(n_groups) + 1) / n_groups

# Otherwise the first row will not fall in the first region (because they are 0) 
# and last rows will not fall in the last region, because they reach 1.0
lower_bounds[0] = -np.inf
upper_bounds[-1] = np.inf

row_in_region = []

for lower_bound, upper_bound in zip(lower_bounds, upper_bounds):
    row_in_region.append(
        np.logical_and(
            relative_cumsum_along_axis > lower_bound, relative_cumsum_along_axis <= upper_bound
        )
    )

row_in_region = np.array(row_in_region).T
final = row_in_region.astype(float)

# find initial region for each row
initial_regions = np.apply_along_axis(np.flatnonzero, 1, row_in_region).flatten()

# find transitions between regions
region_borders = np.flatnonzero(np.diff(initial_regions))

divide_y_positions = []

# finds overlap in transition region
for previous_region, (ventral_row, upper_bound) in enumerate(
    zip(region_borders, upper_bounds)
):
    dorsal_row = ventral_row + 1
    next_region = previous_region + 1
    a, b = relative_cumsum_along_axis[ventral_row], relative_cumsum_along_axis[dorsal_row]
    diff = b - a
    to_a = upper_bound - a
    fraction_to_a = to_a / diff
    fraction_to_b = 1 - fraction_to_a

    final[dorsal_row, previous_region] = fraction_to_a
    final[dorsal_row, next_region] = fraction_to_b
final = final.T
final = final * vector
# convert to list of vectors
final = [final[n, :] for n in range(final.shape[0])]

In [88]:
from eitprocessing.roi_selection.gridselection import GridSelection, DivisionMethod

grid_select = GridSelection(2,2,DivisionMethod.physiological, DivisionMethod.physiological)

In [89]:
matrices = grid_select.find_grid(matrix)

In [94]:
a = matrices[0]